### 네이버 연관 검색어 수집
- 정적(static) 웹페이지 데이터 수집 
- BeautifulSoup을 이용하여 HTML 문자열 데이터 parsing

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
# 1. 웹페이지 분석 : URL

In [3]:
query = "삼성전자"
url = f"https://search.naver.com/search.naver?query={query}"
print(url)

https://search.naver.com/search.naver?query=삼성전자


In [4]:
# 2. request(URL) > response : str(html)

In [5]:
response = requests.get(url)
response

<Response [200]>

In [6]:
response.text[:250]

'<!doctype html> <html lang="ko"><head> <meta charset="utf-8"> <meta name="referrer" content="always">  <meta name="format-detection" content="telephone=no,address=no,email=no"> <meta property="og:title" content="삼성전자 : 네이버 통합검색"/> <meta property="og:'

In [7]:
# 3. str(html) > bs object

In [8]:
dom = BeautifulSoup(response.text, "html.parser")
type(dom)

bs4.BeautifulSoup

In [9]:
# 4. bs object > .select(css-selector), .select_one(css-selector) > str(text)

In [10]:
# 10개의 엘리먼트들 선택
# select() : list(Tag, Tag)
# select_one() : Tag
elements = dom.select(".lst_related_srch > .item")
len(elements)

10

In [11]:
element = elements[0]
keyword = element.select_one(".tit").text
keyword

'삼성전자주가'

In [12]:
link = element.select_one("a").get("href")
link

'?where=nexearch&ssc=tab.nx.all&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90%EC%A3%BC%EA%B0%80&sm=tab_she&qdt=0'

In [13]:
element.text.strip()

'삼성전자주가'

In [14]:
# 각각의 엘리먼트에서 text 데이터 수집

In [15]:
keywords = []
for element in elements:
    keyword = element.text.strip()
    keywords.append(keyword)
print(keywords)

['삼성전자주가', '삼성전자주식', '삼성전자 배당금', '삼성전자 성과급', '오늘 삼성전자 주가', '삼성전자서비스', '삼성전자서비스센타', '삼성전자주식가격', '삼성전자주가지수', '삼성전자 격려금']


In [16]:
keywords = [element.text.strip() for element in elements]
print(keywords)

['삼성전자주가', '삼성전자주식', '삼성전자 배당금', '삼성전자 성과급', '오늘 삼성전자 주가', '삼성전자서비스', '삼성전자서비스센타', '삼성전자주식가격', '삼성전자주가지수', '삼성전자 격려금']


In [17]:
# 5. str(text) > DataFrame

In [18]:
df = pd.DataFrame({"keywords": keywords})
df["query"] = query
df.tail(2)

keywords query
8  삼성전자주가지수  삼성전자
9  삼성전자 격려금  삼성전자

In [19]:
# 현재 시간 데이터 추가

In [20]:
from datetime import datetime

In [21]:
now = datetime.now()
now = now.strftime("%Y-%m-%d %H:%M")
now

'2024-02-26 13:54'

In [22]:
df["date_time"] = now
df.tail(2)

keywords query         date_time
8  삼성전자주가지수  삼성전자  2024-02-26 13:54
9  삼성전자 격려금  삼성전자  2024-02-26 13:54

In [23]:
# query를 입력하면 데이터 프레임을 출력하는 함수
def naver_relate_keyword(query):
    
    url = f"https://search.naver.com/search.naver?query={query}"
    response = requests.get(url)
    dom = BeautifulSoup(response.text, "html.parser")
    elements = dom.select(".lst_related_srch > .item")
    keywords = [element.text.strip() for element in elements]
    
    df = pd.DataFrame({"keywors": keywords})
    df["query"] = query
    
    now = datetime.now()
    now = now.strftime("%Y-%m-%d %H:%M")
    df["date_time"] = now
    
    return df

In [24]:
query = "삼성전자"
df = naver_relate_keyword(query)
df.tail(2)

keywors query         date_time
8  삼성전자주가지수  삼성전자  2024-02-26 13:54
9  삼성전자 격려금  삼성전자  2024-02-26 13:54

In [25]:
dfs = []
queries = ["삼성전자", "LG전자"]

for query in queries:
    print(query, end=" ")
    df = naver_relate_keyword(query)
    dfs.append(df)
    
result = pd.concat(dfs, ignore_index=True)
result

삼성전자 LG전자 

keywors query         date_time
0            삼성전자주가  삼성전자  2024-02-26 13:54
1            삼성전자주식  삼성전자  2024-02-26 13:54
2          삼성전자 배당금  삼성전자  2024-02-26 13:54
3          삼성전자 성과급  삼성전자  2024-02-26 13:54
4        오늘 삼성전자 주가  삼성전자  2024-02-26 13:54
5           삼성전자서비스  삼성전자  2024-02-26 13:54
6         삼성전자서비스센타  삼성전자  2024-02-26 13:54
7          삼성전자주식가격  삼성전자  2024-02-26 13:54
8          삼성전자주가지수  삼성전자  2024-02-26 13:54
9          삼성전자 격려금  삼성전자  2024-02-26 13:54
10          lg전자 주가  LG전자  2024-02-26 13:54
11        lg전자 베스트샵  LG전자  2024-02-26 13:54
12  lg전자 서비스센터 전화번호  LG전자  2024-02-26 13:54
13         lg 전자레인지  LG전자  2024-02-26 13:54
14        lg전자 고객센터  LG전자  2024-02-26 13:54
15         lg전자 성과급  LG전자  2024-02-26 13:54
16           lg전자렌지  LG전자  2024-02-26 13:54
17         lg전자 구독료  LG전자  2024-02-26 13:54
18          lg전자 as  LG전자  2024-02-26 13:54
19          lg전자 주식  LG전자  2024-02-26 13:54